In [1]:
# Dependencies
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
import requests
from pprint import pprint
from config import crimekey
from config import censuskey
from census import Census
c = Census(censuskey, year=2013)

In [2]:
# Read CSVs
edupath = "Resources/cc_institution_details_clean.csv"
#crimepath = 

edu = pd.read_csv(edupath,encoding='utf-8')
#crime = pd.read_csv(crimepath)

edu.head()


,unitid,chronname,city,state,level,control,basic,hbcu,flagship,long_x,...,vsa_grad_after6_transfer,vsa_grad_elsewhere_after6_transfer,vsa_enroll_after6_transfer,vsa_enroll_elsewhere_after6_transfer,similar,state_sector_ct,carnegie_ct,counted_pct,nicknames,cohort_size
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,-86.568502,...,36.4,5.6,17.2,11.1,232937|100724|405997|113607|139533|144005|2285...,13.0,386.0,99.7|07,NaN,882.0
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,-86.809170,...,NaN,NaN,NaN,NaN,196060|180461|201885|145600|209542|236939|1268...,13.0,106.0,56.0|07,UAB,1376.0
2,100690,Amridge University,Montgomery,Alabama,4-year,Private not-for-profit,Baccalaureate Colleges--Arts & Sciences,NaN,NaN,-86.174010,...,NaN,NaN,NaN,NaN,217925|441511|205124|247825|197647|221856|1353...,16.0,252.0,100.0|07,NaN,3.0
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,-86.638420,...,0.0,0,0.0,0.0,232186|133881|196103|196413|207388|171128|1900...,13.0,106.0,43.1|07,UAH,759.0
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,-86.295677,...,NaN,NaN,NaN,NaN,100654|232937|242617|243197|144005|241739|2354...,13.0,386.0,88.0|07,ASU,1351.0


In [3]:
edu.columns

Index(['unitid', 'chronname', 'city', 'state', 'level', 'control', 'basic',
       'hbcu', 'flagship', 'long_x', 'lat_y', 'site', 'student_count',
       'awards_per_value', 'awards_per_state_value', 'awards_per_natl_value',
       'exp_award_value', 'exp_award_state_value', 'exp_award_natl_value',
       'exp_award_percentile', 'ft_pct', 'fte_value', 'fte_percentile',
       'med_sat_value', 'med_sat_percentile', 'aid_value', 'aid_percentile',
       'endow_value', 'endow_percentile', 'grad_100_value',
       'grad_100_percentile', 'grad_150_value', 'grad_150_percentile',
       'pell_value', 'pell_percentile', 'retain_value', 'retain_percentile',
       'ft_fac_value', 'ft_fac_percentile', 'vsa_year',
       'vsa_grad_after4_first', 'vsa_grad_elsewhere_after4_first',
       'vsa_enroll_after4_first', 'vsa_enroll_elsewhere_after4_first',
       'vsa_grad_after6_first', 'vsa_grad_elsewhere_after6_first',
       'vsa_enroll_after6_first', 'vsa_enroll_elsewhere_after6_first',
       'vsa

In [4]:
edu_df = edu[["state","level","basic","level","student_count"]]
edu_df.head()

,state,level,basic,level,student_count
0,Alabama,4-year,Masters Colleges and Universities--larger prog...,4-year,4051
1,Alabama,4-year,Research Universities--very high research acti...,4-year,11502
2,Alabama,4-year,Baccalaureate Colleges--Arts & Sciences,4-year,322
3,Alabama,4-year,Research Universities--very high research acti...,4-year,5696
4,Alabama,4-year,Masters Colleges and Universities--larger prog...,4-year,5356


In [5]:
# API set up for nation-wide 2019

baseurl = "http://api.usa.gov/crime/fbi/sapi/"

#URL = ("https://api.usa.gov/crime/fbi/sapi/api/summarized/state/AL/violent-crime/2019/2019?API_KEY=" + crimekey)

stateAbbs = ['AL', 'AK', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS',
            'KY', 'LA', 'MA', 'ME', 'MD', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 
            'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
statedict = {}
record = 1

# loop through all states and pull data for each
for item in stateAbbs:
    query_url = "https://api.usa.gov/crime/fbi/sapi/api/summarized/state/" + item + "/violent-crime/2019/2019?API_KEY=" + crimekey
    # Get response into JSON
    stateresponse = requests.get(query_url)
    statejson = stateresponse.json()
    x = len(statejson['results'])
    # Log each state and account for exceptions
    try:
        # Collect crime data and put it into a dictionary
        actuals = 0
        for i in range(x):
            actuals = actuals + statejson['results'][i]['actual']
            statedict[item] = [actuals]
        print(f"Processing record {record} | {item}")
        record += 1
        
            
    # Exception if state data isn't found
    except:
        print(f"Data not found for {item}. Skipping...")

print("----------Job complete!----------")

Processing record 1 | AL
Processing record 2 | AK
Processing record 3 | AR
Processing record 4 | AZ
Processing record 5 | CA
Processing record 6 | CO
Processing record 7 | CT
Processing record 8 | DE
Processing record 9 | DC
Processing record 10 | FL
Processing record 11 | GA
Processing record 12 | HI
Processing record 13 | ID
Processing record 14 | IL
Processing record 15 | IN
Processing record 16 | IA
Processing record 17 | KS
Processing record 18 | KY
Processing record 19 | LA
Processing record 20 | MA
Processing record 21 | ME
Processing record 22 | MD
Processing record 23 | MI
Processing record 24 | MN
Processing record 25 | MO
Processing record 26 | MS
Processing record 27 | MT
Processing record 28 | NC
Processing record 29 | ND
Processing record 30 | NE
Processing record 31 | NH
Processing record 32 | NJ
Processing record 33 | NM
Processing record 34 | NV
Processing record 35 | NY
Processing record 36 | OH
Processing record 37 | OK
Processing record 38 | OR
Processing record 39 

In [69]:
# convert crime data into dataframe
# Convert the dictionary into a dataframe
statedf = pd.DataFrame.from_dict(statedict, orient='index')
statedf.head()
statedf.to_csv("Output/statedf.csv", index=False, header=True)

In [7]:
# API set up for crime nation-wide 2019

baseurl = "http://api.usa.gov/crime/fbi/sapi/"
#URL = ("https://api.usa.gov/crime/fbi/sapi/api/summarized/state/AL/violent-crime/2019/2019?API_KEY=" + crimekey)

#getdatasets = requests.get(URL)
#datasets_json = getdatasets.json()
#pprint(datasets_json)

stateAbbs = ['AL', 'AK', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS',
            'KY', 'LA', 'MA', 'ME', 'MD', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 
            'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
statedict = {}
record = 1

# loop through all states and pull data for each
for item in stateAbbs:
    query_url = "https://api.usa.gov/crime/fbi/sapi/api/summarized/state/" + item + "/violent-crime/2019/2019?API_KEY=" + crimekey
   
    # Get response into JSON
    stateresponse = requests.get(query_url)
    statejson = stateresponse.json()
    pprint(statejson)

{'pagination': {'count': 352, 'page': 0, 'pages': 18, 'per_page': 20},
 'results': [{'actual': 209,
              'cleared': 62,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'AL0010000',
              'state_abbr': 'AL'},
             {'actual': 383,
              'cleared': 3,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'AL0010100',
              'state_abbr': 'AL'},
             {'actual': 0,
              'cleared': 0,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'AL0010300',
              'state_abbr': 'AL'},
             {'actual': 122,
              'cleared': 18,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'AL0010400',
              'state_abb

{'pagination': {'count': 283, 'page': 0, 'pages': 15, 'per_page': 20},
 'results': [{'actual': 19,
              'cleared': 4,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'AR0010000',
              'state_abbr': 'AR'},
             {'actual': 29,
              'cleared': 10,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'AR0010100',
              'state_abbr': 'AR'},
             {'actual': 101,
              'cleared': 28,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'AR0010200',
              'state_abbr': 'AR'},
             {'actual': 0,
              'cleared': 0,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'AR0010300',
              'state_abbr'

{'pagination': {'count': 740, 'page': 0, 'pages': 37, 'per_page': 20},
 'results': [{'actual': 633,
              'cleared': 325,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'CA0010000',
              'state_abbr': 'CA'},
             {'actual': 162,
              'cleared': 66,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'CA0010100',
              'state_abbr': 'CA'},
             {'actual': 40,
              'cleared': 13,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'CA0010200',
              'state_abbr': 'CA'},
             {'actual': 618,
              'cleared': 146,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'CA0010300',
              'stat

{'pagination': {'count': 106, 'page': 0, 'pages': 6, 'per_page': 20},
 'results': [{'actual': 36,
              'cleared': 12,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'CT0000200',
              'state_abbr': 'CT'},
             {'actual': 4,
              'cleared': 4,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'CT0000400',
              'state_abbr': 'CT'},
             {'actual': 16,
              'cleared': 8,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'CT0000700',
              'state_abbr': 'CT'},
             {'actual': 6,
              'cleared': 5,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'CT0000900',
              'state_abbr': 'C

{'pagination': {'count': 3, 'page': 0, 'pages': 1, 'per_page': 20},
 'results': [{'actual': 0,
              'cleared': 0,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'DC0011300',
              'state_abbr': 'DC'},
             {'actual': 6896,
              'cleared': 3571,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'DCMPD0000',
              'state_abbr': 'DC'},
             {'actual': 507,
              'cleared': 314,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'DCMTP0000',
              'state_abbr': 'DC'}]}
{'pagination': {'count': 678, 'page': 0, 'pages': 34, 'per_page': 20},
 'results': [{'actual': 738,
              'cleared': 372,
              'data_range': None,
              'data_year': 2019,
              'offense': 

...

{'pagination': {'count': 421, 'page': 0, 'pages': 22, 'per_page': 20},
 'results': [{'actual': 145,
              'cleared': 114,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'TN0010000',
              'state_abbr': 'TN'},
             {'actual': 47,
              'cleared': 39,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'TN0010100',
              'state_abbr': 'TN'},
             {'actual': 16,
              'cleared': 13,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'TN0010200',
              'state_abbr': 'TN'},
             {'actual': 113,
              'cleared': 76,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'TN0010300',
              'state_

{'pagination': {'count': 130, 'page': 0, 'pages': 7, 'per_page': 20},
 'results': [{'actual': 4,
              'cleared': 2,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'UT0010000',
              'state_abbr': 'UT'},
             {'actual': 12,
              'cleared': 4,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'UT0020000',
              'state_abbr': 'UT'},
             {'actual': 41,
              'cleared': 15,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'UT0020100',
              'state_abbr': 'UT'},
             {'actual': 14,
              'cleared': 8,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'UT0020200',
              'state_abbr': '

{'pagination': {'count': 86, 'page': 0, 'pages': 5, 'per_page': 20},
 'results': [{'actual': 0,
              'cleared': 0,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'VT0010000',
              'state_abbr': 'VT'},
             {'actual': 1,
              'cleared': 0,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'VT0010100',
              'state_abbr': 'VT'},
             {'actual': 16,
              'cleared': 13,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'VT0010200',
              'state_abbr': 'VT'},
             {'actual': 5,
              'cleared': 3,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'VT0010300',
              'state_abbr': 'VT'

{'pagination': {'count': 436, 'page': 0, 'pages': 22, 'per_page': 20},
 'results': [{'actual': 21,
              'cleared': 21,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'WI0010000',
              'state_abbr': 'WI'},
             {'actual': 9,
              'cleared': 4,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'WI0010100',
              'state_abbr': 'WI'},
             {'actual': 4,
              'cleared': 0,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'WI0010200',
              'state_abbr': 'WI'},
             {'actual': 22,
              'cleared': 19,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'WI0020000',
              'state_abbr': 

{'pagination': {'count': 55, 'page': 0, 'pages': 3, 'per_page': 20},
 'results': [{'actual': 6,
              'cleared': 2,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'WY0010000',
              'state_abbr': 'WY'},
             {'actual': 58,
              'cleared': 27,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'WY0010100',
              'state_abbr': 'WY'},
             {'actual': 3,
              'cleared': 3,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'WY0010200',
              'state_abbr': 'WY'},
             {'actual': 3,
              'cleared': 1,
              'data_range': None,
              'data_year': 2019,
              'offense': 'violent-crime',
              'ori': 'WY0020000',
              'state_abbr': 'WY'

In [ ]:
# API Setup to get crime data of CA only in 2020
#base_url = "http://api.usa.gov/crime/fbi/sapi/"
#c = Census(censuskey, year=2019)

#URL = ("https://api.usa.gov/crime/fbi/sapi/api/summarized/state/CA/violent-crime/2020/2020?API_KEY=" + crimekey)

#getdatasets = requests.get(URL)
#datasets_json = getdatasets.json()
#pprint(datasets_json)

In [66]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

census_pd.head()

,State,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,01,Alabama,4799277.0,38.1,43253.0,23680.0,870631.0,18.140878,5.040968
1,02,Alaska,720316.0,33.6,70760.0,32651.0,69514.0,9.650487,4.572854
2,04,Arizona,6479703.0,36.3,49774.0,25358.0,1131901.0,17.468409,4.882323
3,05,Arkansas,2933369.0,37.5,40768.0,22170.0,547328.0,18.658682,4.132961
4,06,California,37659181.0,35.4,61094.0,29527.0,5885417.0,15.628107,5.758662


In [68]:
census_pd.to_csv("Output/census_raw.csv")

In [9]:
# create df of population
population = census_pd[["Name","Population"]]
population = population.rename(columns={"Name": "state"})
population.head()

,state,Population
0,Alabama,4799277.0
1,Alaska,720316.0
2,Arizona,6479703.0
3,Arkansas,2933369.0
4,California,37659181.0


In [10]:
# create df education graduation
edu_graduate = edu_df[["state","student_count"]]
edu_graduate.head()

,state,student_count
0,Alabama,4051
1,Alabama,11502
2,Alabama,322
3,Alabama,5696
4,Alabama,5356


In [11]:
# merge to have df ab population and graduation then find the gradu rate column
edu_rate_df = pd.merge(edu_graduate,population,how = "left")
edu_rate_df = edu_rate_df.rename(columns={"student_count": "number of graduates"})
edu_rate_df.head()

,state,number of graduates,Population
0,Alabama,4051,4799277.0
1,Alabama,11502,4799277.0
2,Alabama,322,4799277.0
3,Alabama,5696,4799277.0
4,Alabama,5356,4799277.0


In [13]:
#calculate the graduate % and add into new column
edu_rate_df["% graduate"] = edu_rate_df["number of graduates"]/edu_rate_df["Population"]*100
edu_rate_df.head()

,state,number of graduates,Population,% graduate
0,Alabama,4051,4799277.0,0.084409
1,Alabama,11502,4799277.0,0.239661
2,Alabama,322,4799277.0,0.006709
3,Alabama,5696,4799277.0,0.118685
4,Alabama,5356,4799277.0,0.111600


In [81]:
edu_groupby = edu_rate_df.groupby(["state"])
edu_groupby.mean()
edu_groupby.head()

,state,number of graduates,Population,% graduate
0,Alabama,4051,4799277.0,0.084409
1,Alabama,11502,4799277.0,0.239661
2,Alabama,322,4799277.0,0.006709
3,Alabama,5696,4799277.0,0.118685
4,Alabama,5356,4799277.0,0.111600
...,...,...,...,...
3008,Wyoming,1788,570134.0,0.313610
3009,Wyoming,4780,570134.0,0.838399
3010,Wyoming,1880,570134.0,0.329747
3690,-78.822371,38763,NaN,NaN


In [70]:
edu_rate_df.to_csv("Output/edu_rate_df.csv", index=False, header=True)

In [72]:
# convert the statenames in to one format of AL to do merge to get master dataF

stateNames = {
  "AL" : "Alabama",
  "AK" : "Alaska",
  "AZ" : "Arizona",
  "AR" : "Arkansas",
  "CA" : "California",
  "CO" : "Colorado",
  "CT" : "Connecticut",
  "DE" : "Delaware",
  "FL" : "Florida",
  "GA" : "Georgia",
  "HI" : "Hawaii",
  "ID" : "Idaho",
  "IL" : "Illinois",
  "IN" : "Indiana",
  "IA" : "Iowa",
  "KS" : "Kansas",
  "KY" : "Kentucky",
  "LA" : "Louisiana",
  "ME" : "Maine",
  "MD" : "Maryland",
  "MA" : "Massachusetts",
  "MI" : "Michigan",
  "MN" : "Minnesota",
  "MS" : "Mississippi",
  "MO" : "Missouri",
  "MT" : "Montana",
  "NE" : "Nebraska",
  "NV" : "Nevada",
  "NH" : "New Hampshire",
  "NJ" : "New Jersey",
  "NM" : "New Mexico",
  "NY" : "New York",
  "NC" : "North Carolina",
  "ND" : "North Dakota",
  "OH" : "Ohio",
  "OK" : "Oklahoma",
  "OR" : "Oregon",
  "PA" : "Pennsylvania",
  "RI" : "Rhode Island",
  "SC" : "South Carolina",
  "SD" : "South Dakota",
  "TN" : "Tennessee",
  "TX" : "Texas",
  "UT" : "Utah",
  "VT" : "Vermont",
  "VA" : "Virginia",
  "WA" : "Washington",
  "WV" : "West Virginia",
  "WI" : "Wisconsin",
  "WY" : "Wyoming",
  "DC" : "District of Columbia"
}

stateNamedf = pd.DataFrame.from_dict(stateNames, orient='index')
stateNamedf.reset_index()
#stateNamedf

,index,0
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California
5,CO,Colorado
6,CT,Connecticut
7,DE,Delaware
8,FL,Florida
9,GA,Georgia


In [50]:
# Merge the datasets to have the state columns in shortage AL format

#statedf = pd.merge(statedf, stateNamedf, how = "outer")
#edu_crime.rename(columns={'0_x': 'Actuals', '0_y': 'Name'}, inplace=True)

#state_crime = pd.concat([statedf, stateNamedf], axis = 1)
#state_crime.head(10)
#state_crime.reset_index(0)

In [56]:
# Merge the datasets using the sate columns
statemerge = pd.merge(statedf, stateNamedf, left_index=True, right_index=True)
statemerge.rename(columns={'0_x': 'Actuals', '0_y': 'state'}, inplace=True)
statemerge.reset_index()
statemerge.head()

,Actuals,state
AL,1208,Alabama
AK,4742,Alaska
AR,1069,Arkansas
AZ,3379,Arizona
CA,9734,California


In [71]:
statemerge.to_csv("Output/statemerge.csv", index=False, header=True)

In [1]:
# merge two df together to get master_plot_df : continue in Tracie_shortcut file
